# Uninformed Search
*Important:* Remember to run cells to see the results. Some cells rely on cells further up the page being run, so be aware of this as you make modifications. Run each cell as you work your way down the page even if results are already showing. If something isn't working, restart the kernel, and rerun the entire notebook.

## Tower of Hanoi
### Introduction

The Tower of Hanoi is a well known mathematical puzzle from 1883. You are given three pegs which hold a number of circular disks of different sizes. The disks all start on the furthest left peg, and you must move the entire tower to the furthest right peg, one disk at a time, obeying a simple rule: you may not put a disk on top of a smaller disk.

<img src="https://upload.wikimedia.org/wikipedia/commons/0/07/Tower_of_Hanoi.jpeg" align=center />

<p style="text-align: center;">Tower of Hanoi puzzle with 8 disks in its start configuration. Image from <a href="https://commons.wikimedia.org/w/index.php?curid=228623">Wikimedia</a> under <a href="http://creativecommons.org/licenses/by-sa/3.0/" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a> licence.</p>

### Code
There is a Python module in this project which implements the Tower of Hanoi game called `hanoi.py`. You do not need to look at the code – you can if you wish, but I suggest working through this entire notebook once first. It contains a class called `HanoiState` which allows us to represent the state of the puzzle. When you create a new state with no additional arguments, it will create the starting state for a 5-disk puzzle:

In [13]:
from hanoi import HanoiState

state = HanoiState()
print(state)

 1 | |
 2 | |
 3 | |
 4 | |
 5 | |



Notice that numbers are used to represent the size of the disks, with `1` being the smallest. We can provide an argument to the constructor to create a state with a different number of disks (or pegs):

In [14]:
state = HanoiState(disks=3)
print(state)

 1 | |
 2 | |
 3 | |



From any state, we can call the `possible_actions` method to find what actions are available:

In [15]:
print(state.possible_actions())

[(0, 1), (0, 2)]


The `possible_actions` method returns a list of tuples. A single tuple `(x, y)` indicates that it is possible to move a disk from peg `x` to peg `y`, with the first peg being numbered `0`.

We can use this information, or otherwise, to move disks around. We do this using the `next_state(from_peg, to_peg)` method.

In [16]:
state = state.next_state(0, 2)
print(state)

 | | |
 2 | |
 3 | 1



*Important point:* note that HanoiState objects are *immutable*. You cannot change their internal state. We had to write:
```python
state = state.next_state(0, 2)
```

rather than
```python
state.next_state(0, 2)
```

Simply calling the method and ignoring the result will not change the original object:

In [17]:
test = HanoiState(disks=3)
test.next_state(0, 2)
print(test)

 1 | |
 2 | |
 3 | |



This is a really useful property when writing search algorithms, because it means we can keep track of which states we've seen before, and we don't have to worry about accidentally changing their internal values later. This property is explained in more detail in [a separate notebook](additional/immutable_state.ipynb), but it will be helpful to understand the basics of the breadth first search algorithm first.

Back to the code. There is a method which we can call to query whether the state is a goal state: i.e. has the game been solved, are all the disks on the furthest right peg?

In [18]:
print(state)
print(state.is_goal_state())

 | | |
 2 | |
 3 | 1

False


Now let's do the full series of moves required to solve the puzzle with 3 disks:

In [19]:
state = state.next_state(0, 1)
state = state.next_state(2, 1)
state = state.next_state(0, 2)
state = state.next_state(1, 0)
state = state.next_state(1, 2)
state = state.next_state(0, 2)
print(state)
print(state.is_goal_state())

 | | 1
 | | 2
 | | 3

True


### Breadth-First Search
Given the starting configuration with all of the disks on the first peg, we'd like to search for the final configuration, with all disks on the rightmost peg. Our real objective here is to find the *path* that actually takes us from one to the other, but as described in the videos, this is something we will easily be able to solve by working backwards once we have found it.

Here is a copy of the breadth-first search algorithm, pseudocode taken from Russel and Norvig (p. 82):<br />
<img src="./additional/Breadth_first_search.png" width=60% /> <br />

And here is an implementation for the Tower of Hanoi problem:

In [20]:
def breadth_first_search():
    state = HanoiState(disks=5)
    frontier = [state]
    explored = []

    current_state = frontier.pop(0)
    while not current_state.is_goal_state():
        explored.append(current_state)
        actions = current_state.possible_actions()
        for action in actions:
            new_state = current_state.next_state(action[0], action[1])
            if new_state not in explored and new_state not in frontier:
                frontier.append(new_state)

        if len(frontier) == 0:
            return None

        current_state = frontier.pop(0)

    return current_state


final_state = breadth_first_search()

if final_state is None:
    print("No solution found...")
else:
    print("Solution found!")
    print(final_state)

Solution found!
 | | 1
 | | 2
 | | 3
 | | 4
 | | 5



Make sure you spend a good amount of time with this code. Remember you can modify it and re-run it to see the results. You could add print statements which show the contents of the frontier at each point in the algorithm, for example.

### Finding The Path
We can make a small modification to this algorithm which will allow us to work backwards from the goal state to the start state, thereby showing us the entire sequence of moves which solves the game.

`HanoiState` objects have an attribute called `parent`. When you create a state using the `next_state` method, the attribute of the new state is set automatically to point at the original state. Here is a demonstration:

In [21]:
state = HanoiState(disks=3)
state = state.next_state(0, 2)
print("Here is the value of state:")
print(state)
print("Here is the value of state.parent")
print(state.parent)
print("Here is the value of state.parent.parent")
print(state.parent.parent)

Here is the value of state:
 | | |
 2 | |
 3 | 1

Here is the value of state.parent
 1 | |
 2 | |
 3 | |

Here is the value of state.parent.parent
None


So each state has a reference to its parent state, all the way back to the start state, whose `parent` attribute is set to `None`. This means that once we've found the goal state, we can easily backtrack by following the chain with something like a while loop. 

Here is a new version of `breadth_first_search()` which prints the entire path once the solution has been found. 
* Read the code carefully to see how it iterates through the states backwards, adding them to a list, and then prints this list backwards to obtain the result in the correct order. 
* Notice that the code also keeps track of some simple metrics that allow us to compare how efficiently the algorithm is performing: it will print the total number of states that it generates, and the total number it explores (the number of unique states it generates).

In [22]:
def breadth_first_search():
    state = HanoiState(disks=5)
    frontier = [state]
    explored = []
    generated = 0

    current_state = frontier.pop(0)
    while not current_state.is_goal_state():
        explored.append(current_state)
        actions = current_state.possible_actions()
        for action in actions:
            generated += 1
            new_state = current_state.next_state(action[0], action[1])
            if new_state not in explored and new_state not in frontier:
                frontier.append(new_state)

        if len(frontier) == 0:
            print("No solution found")
            return

        current_state = frontier.pop(0)

    print("Solution found!")
    print(f"Explored {len(explored)} states")
    print(f"Generated {generated} states")
    print()

    final_path = []
    while current_state.parent is not None:
        final_path.append(current_state)
        current_state = current_state.parent

    final_path.append(current_state)

    for state in reversed(final_path):
        if state.action is not None:
            print(f"Move disk from peg {state.action[0]} to {state.action[1]}")
        print(state)
    print(f"Total {len(final_path)-1} steps")

breadth_first_search()

Solution found!
Explored 232 states
Generated 694 states

 1 | |
 2 | |
 3 | |
 4 | |
 5 | |

Move disk from peg 0 to 2
 | | |
 2 | |
 3 | |
 4 | |
 5 | 1

Move disk from peg 0 to 1
 | | |
 | | |
 3 | |
 4 | |
 5 2 1

Move disk from peg 2 to 1
 | | |
 | | |
 3 | |
 4 1 |
 5 2 |

Move disk from peg 0 to 2
 | | |
 | | |
 | | |
 4 1 |
 5 2 3

Move disk from peg 1 to 0
 | | |
 | | |
 1 | |
 4 | |
 5 2 3

Move disk from peg 1 to 2
 | | |
 | | |
 1 | |
 4 | 2
 5 | 3

Move disk from peg 0 to 2
 | | |
 | | |
 | | 1
 4 | 2
 5 | 3

Move disk from peg 0 to 1
 | | |
 | | |
 | | 1
 | | 2
 5 4 3

Move disk from peg 2 to 1
 | | |
 | | |
 | | |
 | 1 2
 5 4 3

Move disk from peg 2 to 0
 | | |
 | | |
 | | |
 2 1 |
 5 4 3

Move disk from peg 1 to 0
 | | |
 | | |
 1 | |
 2 | |
 5 4 3

Move disk from peg 2 to 1
 | | |
 | | |
 1 | |
 2 3 |
 5 4 |

Move disk from peg 0 to 2
 | | |
 | | |
 | | |
 2 3 |
 5 4 1

Move disk from peg 0 to 1
 | | |
 | | |
 | 2 |
 | 3 |
 5 4 1

Move disk from peg 2 to 1
 | | |
 | 1 

For the Tower of Hanoi, it is known that for $n$ disks, the shortest path from the start state to the end state is $2^n-1$ moves (a total of $2^n$ states including the start state). As the result at the end of the code shows, our breadth first search found a sequence of moves which was optimal. (Notice that `len(final_path)` is the total number of states including the start state, so `len(final_path)-1` is the number of moves.)

### Your Turn
If you haven't already, try modifying the code above to produce different results. What happens if you change the number of disks, or even the number of pegs? The search space can get large quite quickly, so don't forget you can press the stop button on the toolbar to terminate a cell that has been running for too long.

Breadth first search is guaranteed to find a solution with the minimum number of steps, because it checks all solutions of length $n$ before it starts trying solutions of length $n+1$. However, storing all of the unexplored states (the frontier) uses a lot of memory.

We also learned about *depth first search*. In a breadth first search, the shallowest node in the frontier is expanded first, whereas in a depth first search, it is the deepest.

**Exercise:** Write your own code in the cell below to perform depth first search. It can be done with a surprisingly small modification to the breadth first graph search code. This is described in more detail in the textbook. It can also be implemented with a recursive function, but this will likely be more work. 

Answer the following questions of your depth first search solution:
* Is the result of optimal length (in number of steps)? 
* Does it generate more or fewer states, using the same metrics as the previous code? 
* When do you think we'd be likely to use depth first search, and when would we use breadth first?
* If this version is not optimal, what modifications would be necessary to find an optimal result? 

In [23]:
def depth_first_search():
    # write your code here!
    pass

If you are stuck, you can [click here](additional/depth_first.ipynb) to see a solution to the depth first search problem.